EDA

In [1]:
%reload_ext autoreload
%autoreload 1
import joblib
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
import auxiliary.lists as aux_lists
import auxiliary.transformers as tr
import auxiliary.eda_functions as eda
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
%aimport auxiliary.lists
%aimport auxiliary.transformers
%aimport auxiliary.eda_functions


In [71]:
# applicattion_test=pl.read_csv('data/application_test.csv')
application_train=pl.read_csv('data/application_train.csv')
buereau_balance=pl.read_csv('data/bureau_balance.csv')
buereau=pl.read_csv('data/bureau.csv')
installments_payments=pl.read_csv('data/installments_payments.csv')
POS_CASH_balance=pl.read_csv('data/POS_CASH_balance.csv')
previous_application=pl.read_csv('data/previous_application.csv')
id_and_target=["SK_ID_CURR", "TARGET"]

In [51]:
buereau_num_cols = (
    buereau.select([pl.col(pl.FLOAT_DTYPES), pl.col(pl.INTEGER_DTYPES)])
    .drop(columns=id_and_target)
    .columns
)
buereau_num_cols

In [44]:
buereau_agg = pl.DataFrame(application_train["SK_ID_CURR"])

for col in buereau_num_cols:
    buereau_agg = buereau_agg.join(
        buereau.group_by(pl.col("SK_ID_CURR")).agg(pl.col(col).mean().suffix("_mean")),
        on="SK_ID_CURR",how='left'
    )

for col in buereau_num_cols:
    buereau_agg = buereau_agg.join(
        buereau.group_by(pl.col("SK_ID_CURR")).agg(pl.col(col).sum().suffix("_sum")),
        on="SK_ID_CURR",how='left'
    )

for col in buereau_num_cols:
    buereau_agg = buereau_agg.join(
        buereau.group_by(pl.col("SK_ID_CURR")).agg(pl.col(col).min().suffix("_min")),
        on="SK_ID_CURR",how='left'
    )

for col in buereau_num_cols:
    buereau_agg = buereau_agg.join(
        buereau.group_by(pl.col("SK_ID_CURR")).agg(pl.col(col).max().suffix("_max")),
        on="SK_ID_CURR",how='left'
    )

for col in buereau_num_cols:
    buereau_agg = buereau_agg.join(
        buereau.group_by(pl.col("SK_ID_CURR")).agg(pl.col(col).std().suffix("_std")),
        on="SK_ID_CURR",how='left'
    )

In [49]:
buereau_agg=buereau_agg.join(application_train[id_and_target],on='SK_ID_CURR',how='left')

In [72]:
test_bureu_aggs = eda.test_with_catboost_crossval(
    buereau_agg.drop(columns=id_and_target), buereau_agg["TARGET"]
)

In [3]:
application_train, application_val = train_test_split(
    application_train,
    train_size=0.33,
    stratify=application_train["TARGET"],
    random_state=1,
)

In [4]:
application_train['TARGET'].value_counts()

TARGET,counts
i64,u32
1,8192
0,93286


#### Duplicates

In [5]:
application_train['SK_ID_CURR'].is_duplicated().any()

False

#### Missing Values


In [6]:
pl.DataFrame(
    {
        "missing_fraction": application_train.null_count().transpose().to_series()
        / len(application_train),
        "feature": application_train.columns,
    }
).sort("missing_fraction",descending=True)[:10]

missing_fraction,feature
f64,str
0.700221,"""COMMONAREA_AVG…"
0.700221,"""COMMONAREA_MOD…"
0.700221,"""COMMONAREA_MED…"
0.695234,"""NONLIVINGAPART…"
0.695234,"""NONLIVINGAPART…"
0.695234,"""NONLIVINGAPART…"
0.684749,"""FONDKAPREMONT_…"
0.684542,"""LIVINGAPARTMEN…"
0.684542,"""LIVINGAPARTMEN…"


In [7]:
string_cols=application_train.select(pl.col(pl.Utf8)).columns

### Feature importance

In [21]:
initial_test=eda.test_with_catboost(application_train,application_val,string_cols)
print(initial_test['report'])
print(initial_test['roc_auc'])
display(initial_test['features'][:10])

              precision    recall  f1-score   support

           0       0.96      0.73      0.83    189400
           1       0.17      0.64      0.27     16633

    accuracy                           0.72    206033
   macro avg       0.57      0.69      0.55    206033
weighted avg       0.90      0.72      0.78    206033



feature,importance
str,f64
"""EXT_SOURCE_3""",12.654869
"""EXT_SOURCE_2""",9.24611
"""EXT_SOURCE_1""",5.942974
"""AMT_CREDIT""",5.273107
"""AMT_GOODS_PRIC…",5.002266
"""AMT_ANNUITY""",4.473649
"""DAYS_BIRTH""",4.395158
"""DAYS_EMPLOYED""",3.758365
"""DAYS_ID_PUBLIS…",2.751794


### Common Sense Removal

In [9]:
application_edited_train = application_train.clone()
application_edited_val = application_val.clone()
cols_to_remove=[]

#### Home Feature Elimination

In [10]:
home_base_features = list(set([col.rsplit("_", 1)[0] for col in aux_lists.home_cols]))
best_home_features = []

for feature in home_base_features:
    best_home_features.append(
        initial_test["features"]
        .filter(pl.col("feature").str.starts_with(feature))
        .sort("importance")["feature"][0]
    )

cols_to_remove.extend([col for col in aux_lists.home_cols if col not in best_home_features])

### Col cats

In [11]:
application_train.filter(pl.col('FLAG_OWN_CAR')=="Y")['OWN_CAR_AGE'].null_count()

3

In [12]:
cols_to_remove.append('FLAG_OWN_CAR')

In [13]:
application_edited_train = application_edited_train.drop(cols_to_remove)

application_edited_val = application_edited_val.drop(cols_to_remove)

string_cols_reduced = application_edited_train.select(pl.col(pl.Utf8)).columns

test_home_removal = eda.test_with_catboost(
    application_edited_train, application_edited_val, string_cols_reduced
)

print(test_home_removal["report"])

              precision    recall  f1-score   support

           0       0.96      0.73      0.83    189400
           1       0.17      0.65      0.27     16633

    accuracy                           0.72    206033
   macro avg       0.57      0.69      0.55    206033
weighted avg       0.90      0.72      0.79    206033



### Variance

In [14]:
home_target_encoders = {}
application_edited_encoded=application_edited_train.clone()
for col in string_cols_reduced:
    home_target_encoders[col] = tr.TargetMeanOrderedLabeler(how="label")
    application_edited_encoded = application_edited_encoded.with_columns(
        home_target_encoders[col]
        .fit_transform(
            application_edited_encoded[col], application_edited_encoded["TARGET"]
        )
        .alias(col)
    )

In [15]:
feature_variance = pl.DataFrame(
    {
        "feature": application_edited_encoded.columns,
        "variance": np.var(MinMaxScaler().fit_transform(application_edited_encoded),axis=0),
    }
)
feature_variance.sort('variance')[:20]

feature,variance
str,f64
"""FLAG_MOBIL""",0.00001
"""FLAG_DOCUMENT_…",0.00001
"""FLAG_DOCUMENT_…",0.00001
"""FLAG_DOCUMENT_…",0.000039
"""FLAG_DOCUMENT_…",0.000108
"""FLAG_DOCUMENT_…",0.000177
"""AMT_INCOME_TOT…",0.00022
"""FLAG_DOCUMENT_…",0.000276
"""FLAG_DOCUMENT_…",0.000384


In [16]:
low_var_fatures=[]
for feature in feature_variance.filter(pl.col('variance')<0.001)['feature']:
    if application_edited_train[feature].n_unique()<10:
        low_var_fatures.append(feature)
        minority_count=application_edited_train[feature].value_counts().sort('counts')['counts'][0]
        print(f'{feature}: minority class count = {minority_count} feature will be removed')

FLAG_MOBIL: minority class count = 1 feature will be removed
FLAG_DOCUMENT_2: minority class count = 4 feature will be removed
FLAG_DOCUMENT_4: minority class count = 11 feature will be removed
FLAG_DOCUMENT_7: minority class count = 18 feature will be removed
FLAG_DOCUMENT_10: minority class count = 1 feature will be removed
FLAG_DOCUMENT_12: minority class count = 1 feature will be removed
FLAG_DOCUMENT_17: minority class count = 28 feature will be removed
FLAG_DOCUMENT_19: minority class count = 62 feature will be removed
FLAG_DOCUMENT_20: minority class count = 52 feature will be removed
FLAG_DOCUMENT_21: minority class count = 39 feature will be removed


In [17]:
application_edited_train=application_edited_train.drop(columns=low_var_fatures)
application_edited_val=application_edited_val.drop(columns=low_var_fatures)

In [23]:
test_var_removal = eda.test_with_catboost(
    application_edited_train, application_edited_val, string_cols_reduced
)

print(test_var_removal["report"])
print(test_var_removal["roc_auc"])

              precision    recall  f1-score   support

           0       0.96      0.73      0.83    189400
           1       0.17      0.64      0.27     16633

    accuracy                           0.72    206033
   macro avg       0.57      0.69      0.55    206033
weighted avg       0.90      0.72      0.78    206033



0.755768296203315
